# Oligo Design

For Nucleaseq, we want to design sequences with the following basic structure:

| Left primer | Left BC | Left buffer | Target$_n$ | Right buffer | Right fill | Right BC | Right primer |
| - | - | - | - | - | - | - | - |

Where Target$_n$ is from the set of all desired modified target sequences. 

The set of Target$_n$'s is specific to each experiment, but there are some relatively standard sets. For example, most experiments will wish to include all single- and double-mismatch sequences. While on the other hand, the PAM structure for each CRISPR variant is different and may require custom sequence generation. And other nucleases may have entirely different needs.

To handle this, we have a number of functions for standard modifications in design.py, which can be called below, while at the same time we have a space explicitly reserved for custom sequence generation: "Custom sequence functions". After the set of target-generation functions is complete, they need added to the "Construct Sequences" section below, in the manner shown by the included examples. Go through this section carefully to verify the set of included sequences is correct.

The "Run parameters" section needs updated according to the experimental requirements, as well. One parameter which needs specified is a list all canonical cut positions along the target sequence. This needs to be a python integer with ".5" after it to indicate the cut position. For instance, 18.5 cuts between python indices 18 and 19. The lower portion of this notebook needs this information to find appropriate primer sequences.

Finally, the user is expected to adjust the primers as necessary to fit their experimental conditions. See the "Replace Primers" section for details.

## Sequence Motifs

### For this notebook, we want the following sequences and modifications:

* Target D - TnpB (IsDra2)
* single mismatches
* double mismatches
* single insertions
* double insertions
* single deletions 
* double deletions
* scanning mismatch regions (using the complement) for 3 bp to 24 bp length regions
* Perfect target with different buffer regions
* Perfect target with various barcodes
* Random negative control seqs
* Various 5N PAM
* single mm and single ins seqs
* single mm and single del seqs

In [1]:
import time
notebook_start_time = time.time()

%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import yaml
import itertools
import random
import math
import logging
import scipy.misc
import editdistance
import numpy as np
from copy import deepcopy
from collections import defaultdict, Counter
from Bio.Seq import Seq
from freebarcodes.seqtools import simple_hamming_distance
import freebarcodes.seqtools as fbseqtools

from nucleaseq import design, seqtools
from nucleaseq.NucleaSeqOligo import NucleaSeqOligo
from nucleaseq.equalmarginalseqs import generate_clean_random_eqmarg_seqs

#import freebarcodes. Need to show jupyter where to find it.
sys.path.append('/home/joneslab/nucleaseq/freebarcodes')

master_seed = 37
random.seed(master_seed)
rand_seeds = [random.randint(0, 100) for i in range(10)]

# Run parameters

In [2]:
# Library features
total_desired_seqs = int(raw_input('What library size do You desire? Defaut: 12472  ') or '12472')
n_err_detect_seqs = int(raw_input('How many negative control sequences? Default: 150  ') or '150')
min_perfect_target_copies = int(raw_input('How many perfect targets do You desire? Default: 50  ') or '50')
bases = 'ACGT'

# Target features
targets_fpath = '/home/joneslab/nucleaseq/resources/targets.yml'#TnpB-targets.yml'
target_name = 'D'
target_pam = 'TTTA' #when you request the PAM, only allow ACGT.
interesting_pams = ['TTTA'] # PAMs to investigate if you're interested in them. Originally: 'NNGAN'
abs_cannonical_cut_sites = [13, 22] #How many bases away from the PAM will the sequence be cut

#Barcode features
barcodes_fpath = '/home/joneslab/nucleaseq/freebarcodes/barcodes/barcodes18-2.txt'#barcodes17-2.txt'#_subset1of3.txt'
barcodes = [line.strip() for line in open(barcodes_fpath)]
bad_substrs = [target_pam, 'ATCAA'] # Forbidden subseqs in buffers or primers (here TnpB TAMs)

# Primer features
min_primer_len = 18  # First length to try. Will go smaller if possible. Adjust this if notebook too slow.
max_primer_len = 25

#Computational limits
nprocs = 2 #20 #Is this the number of processors used for computation

log = logging.getLogger()
log.addHandler(logging.StreamHandler())
log.setLevel(logging.INFO)
log.info('''The provided barcodes file has {} barcodes, 
which could create {} barcode pairs. It is important that the      
amount of barcode pairs is not less than {}- the total_desired_seqs'''.format(
len(barcodes), len(barcodes)/2, total_desired_seqs))

What library size do You desire? Defaut: 12472  
How many negative control sequences? Default: 150  
How many perfect targets do You desire? Default: 50  


The provided barcodes file has 64521 barcodes, 
which could create 32260 barcode pairs. It is important that the      
amount of barcode pairs is not less than 12472- the total_desired_seqs


In [4]:
# Load the file containing the targets used for library generation.
targets = yaml.safe_load(open(targets_fpath))
target = targets[target_name]
target_no_pam = target[len(target_pam):]
target_seeds = [target[:12]]

fudge_factor = 5             # How far from a cannonical cut site are possible cuts
min_buffer_len = 5           # Min length of target-flanking buffers

print 'Consider cut sites within {} bp of pamtarg positions: {}'.format(fudge_factor, abs_cannonical_cut_sites)
print 'Target {} ({} bp): {}'.format(target_name, len(target), target)
print 'Target seeds:'
for target_seed in target_seeds:
    print '    ({} bp): {}'.format(len(target_seed), target_seed)

Consider cut sites within 5 bp of pamtarg positions: [13, 22]
Target D (27 bp): TTTAGTGATAAGTGGAATGCCATGTGG
Target seeds:
    (12 bp): TTTAGTGATAAG


In [5]:
def check_cut_sites(spacer, target_pam, pam_end = '5', cut_sites = [13, 22]):
    #Shows where a cut in the sequence will be made
    outcome = []
    pam_end = str(pam_end)
    if pam_end == '5':
        for ccs in cut_sites:   
            cuts_5 = '[{}]{} X {}'.format(target_pam, spacer[:ccs], spacer[ccs:])
            outcome.append(cuts_5)
    elif pam_end == '3':
        for ccs in cut_sites:   
            cuts_3 = '{} X {}[{}]'.format(spacer[:-ccs], spacer[-ccs:], target_pam)
            outcome.append(cuts_3)
        print('Note: PAM is not flipped')
    elif pam_end != '3' and pam_end != '5':
        raise Exception('''Science knows how to place PAM only on the 3' or 5' end and no other way, 
           so please change pam_end to either 3 or 5''')
    print(outcome[0]+'\n'+outcome[1])


# Static Sequences

In [6]:
# Create primers (cr1,cr2) with a length of 19bp that avoids bad_substrs (PAM sequences, defined above).
# Should be tested experimentally before purchasing (experience shows some primers (D) work better than others (E).
#random.seed(rand_seeds[7])
#cr1 = design.shuffled_equalish_bases(19, bad_substrs)
#cr2 = design.shuffled_equalish_bases(19, bad_substrs)

#Define primers that are known to amplify well:
cr1 = 'AACCGCCGAATAACAGAGT' #NP1
cr2 = str(Seq('AAGAACGCCTCGCACACT').reverse_complement())  #NP2

# create buffers (buffer1,buffer2) that avoid bad_substrs (PAM sequences, defined above)
buffer1 = "CAGAT"
buffer2 = "TGGATC"
#buffer1 = design.get_buffer(5, 'left', target, bad_substrs)
#buffer2 = design.get_buffer(5, 'right', target, bad_substrs)

print 'Temp left/right buffers:', buffer1, buffer2
print 'Temp primer 1: {} bp, {:.1f}% GC  {}'.format(len(cr1), 
                                               100*(cr1.count('C') + cr1.count('G'))/float(len(cr1)),
                                               cr1)
print 'Temp primer 2 (rev_comp): {} bp, {:.1f}% GC  {}'.format(len(cr2), 
                                               100*(cr2.count('C') + cr2.count('G'))/float(len(cr2)),
                                               cr2)

Temp left/right buffers: CAGAT TGGATC
Temp primer 1: 19 bp, 47.4% GC  AACCGCCGAATAACAGAGT
Temp primer 2 (rev_comp): 18 bp, 55.6% GC  AGTGTGCGAGGCGTTCTT


# Generation of Sequences for Pilot Experiment

## Custom sequence functions

In [7]:
#added 'bases' here
def get_pams(interesting_pams, bases = 'ACGT'):
    #################################################################
    # This function generates all possible PAMs by substituting Ns  #
    # with A,C,G and T for all the PAMs included in the input list. #
    #################################################################
    
    assert type(interesting_pams) == list, "The input must be a list."    
    output_pams = [] # Empty list where all the possible PAMs will be added.
    
    # Iterate over each PAM in the input list.
    for pam in interesting_pams:          
        assert set(pam).issubset(set('ACGTN')), "PAMs must be strings that only contain As,Cs,Gs,Ts and/or Ns."
        
        #Find the positions of all Ns in the PAM.
        interesting_positions = []        
        for i,value in enumerate(pam):    
            if value == 'N': interesting_positions.append(i)
                
        #Replace Ns with all possible combinations of A, C, G and T.         
        for n in itertools.product(bases, repeat=len(interesting_positions)):            
            pam_temp = pam
            for i,pos in enumerate(interesting_positions):
                pam_temp = pam_temp[:pos]+n[i]+pam_temp[pos+1:]    
            output_pams.append(pam_temp)  #Add the new PAMs to the list.
            
    return output_pams


def get_interesting_pam_seqs(interesting_pams, target, pam):
    
    #####################################################################################
    # This function outputs all targets with the PAMs from the 'interesting_pams' list. #
    #####################################################################################   
    output_seqs=[]
    pams = get_pams(interesting_pams)
    
    #Replace the indicated PAM in the target with an interesting PAM
    #whether it is at the beginning...
    if target.startswith(pam):
        target=target[len(pam):]
        for pam in pams:
            output_seqs.append(pam+target)
            
    #or the end of the target.      
    elif target.endswith(pam):
        target=target[:(len(target)-len(pam))]
        for pam in pams:
            output_seqs.append(target+pam)  
            
    else:
        raise Exception('A target must end or begin with the PAM.')
    
    return output_seqs


####---------------------------------------------------------------####
def next_different_base(samp):
    samp_bases = set(samp)
    assert len(samp_bases) < 4, samp
    for b in bases:
        if b not in samp_bases:
            return b


def get_one_mm_seq_per_pos(seq):
    output = set()
    for i in range(len(seq)):
        # Choose mismatch to be different from ref base and neighboring bases
        neighborhood = seq[max(i-1, 0):i+2]
        assert len(neighborhood) == 3 or i == 0 or i == len(seq) - 1, (i, neighborhood)
        output.add(seq[:i] + next_different_base(neighborhood) + seq[i+1:])
    return output 


def get_one_ins_seq_per_pos(seq):
    output = set()
    for i in range(len(seq)):
        # Choose insertion to be different from either neighboring base
        neighborhood = seq[max(i-1, 0):i+1]
        assert len(neighborhood) == 2 or i == 0, (i, neighborhood)
        output.add(seq[:i] + next_different_base(neighborhood) + seq[i:])
    return output

    
def get_single_mm_single_del_seqs():
    output = set()
    for del_seq in fbseqtools.get_deletion_seqs(target, 1):
        output.update(get_one_mm_seq_per_pos(del_seq))
    return output
    
    
def get_single_mm_single_ins_seqs():
    output = set()
    for mm_seq in get_one_mm_seq_per_pos(target):
        output.update(get_one_ins_seq_per_pos(mm_seq))
    return output

## Construct sequences

In [8]:
# This cell is the heart of the sequence construction. It is a single cell to 
# guarantee all parts are always performed together, so make reproducible output.

#----------------------------------------------------------------------------------
# Setup the barcode pairs
#----------------------------------------------------------------------------------

def min_dist_to_target_seed(sequence, target_seeds):

    #This function makes sure that the barcodes are not too similar to the seed.
    
    min_dists = []
    for target_seed in target_seeds:
        if len(sequence) > len(target_seed):
            slong, sshort = sequence, target_seed
        else:
            slong, sshort = target_seed, sequence
        slong_rc = seqtools.dna_rev_comp(slong)
        min_dists.append(min(editdistance.eval(sshort, sl[i:i+len(sshort)]) 
                             for i in range(len(slong) - len(sshort) + 1)
                             for sl in [slong, slong_rc]))
    return min(min_dists)

In [9]:
barcodes.sort(key=lambda barcode: min_dist_to_target_seed(barcodes, target_seeds), reverse = True)
# Prefer barcodes disimilar to target seed.

err_barcodes = barcodes[:n_err_detect_seqs]
norm_barcodes = barcodes[n_err_detect_seqs:total_desired_seqs]
log.info('Max / min accepted / min barcode distances to target seed: {} / {} / {}\n'.format(
    min_dist_to_target_seed(err_barcodes[0], target_seeds),
    min_dist_to_target_seed(norm_barcodes[-1], target_seeds),
    min_dist_to_target_seed(barcodes[-1], target_seeds)
))

random.seed(rand_seeds[2])
random.shuffle(norm_barcodes)
random.shuffle(err_barcodes)

Max / min accepted / min barcode distances to target seed: 6 / 6 / 6



In [11]:
barcode_pairs, err_barcode_pairs = [], []
for i in xrange(len(norm_barcodes)):
    bc1 = norm_barcodes[i]
    i2 = (i+1) % len(norm_barcodes)
    bc2_rc = str(Seq(norm_barcodes[i2]).reverse_complement())
    barcode_pairs.append((bc1, bc2_rc))

for i in xrange(len(err_barcodes)):
    bc1 = err_barcodes[i]
    i2 = (i+1) % len(err_barcodes)
    bc2_rc = str(Seq(err_barcodes[i2]).reverse_complement())
    err_barcode_pairs.append((bc1, bc2_rc))

In [12]:
#----------------------------------------------------------------------------------
# Custom sequences
#----------------------------------------------------------------------------------

sequence_set = set()

# Interesting PAMs
interesting_pam_seqs = get_interesting_pam_seqs(interesting_pams,target,target_pam)
sequence_set.update(interesting_pam_seqs)
log.info('Interesting PAM seqs: {}'.format(len(interesting_pam_seqs)))

# Single mm and single del
#single_mm_single_del_sequences = get_single_mm_single_del_seqs()
#log.info('Single mm, Single del seqs: %d' % len(single_mm_single_del_sequences))
#sequence_set.update(single_mm_single_del_sequences)

# Single mm and single ins
#single_mm_single_ins_sequences = get_single_mm_single_ins_seqs()
#log.info('Single mm, Single ins seqs: %d' % len(single_mm_single_ins_sequences))
#sequence_set.update(single_mm_single_ins_sequences)


#----------------------------------------------------------------------------------
# Standard sequences
#----------------------------------------------------------------------------------

#This is static if we dont change the target

# All possible single mismatches and indels
single_mismatches = fbseqtools.get_mismatch_seqs(target, 1)
sequence_set.update(single_mismatches)
log.info('Single mismatch seqs: %d' % len(single_mismatches))

single_insertions = fbseqtools.get_insertion_seqs(target, 1)
sequence_set.update(single_insertions)
log.info('Single insertion seqs: %d' % len(single_insertions))

single_deletions = fbseqtools.get_deletion_seqs(target, 1)
sequence_set.update(single_deletions)
log.info('Single deletion seqs: %d' % len(single_deletions))

# Every stretch of mismatched sequence at each possible position in the target
###NOTE: Altered for TnpB to reduce library size.

c_stretch = set()
#Creates sequences that have a stretch of the same nukleotide (multiple G's one after the other) that are mispaired
for stretch_size in range(2, len(target) + 1):
    complement_stretch = fbseqtools.get_stretch_of_complement_seqs(target, stretch_size)
    #NOTE: #Code to remove PAM from the stretch regions
    #complement_stretch = list(complement_stretch)
    #for i, seq in enumerate(complement_stretch):
    #    complement_stretch[i] = target_pam+complement_stretch[i][len(target_pam):]
    #complement_stretch = set(complement_stretch)
    #End of code to remove PAM
    c_stretch |= complement_stretch
sequence_set.update(c_stretch)
log.info('Complement stretch seqs: %d' % len(c_stretch))


# double mismatches, deletions, and insertions 
double_mismatch_sequences = fbseqtools.get_mismatch_seqs(target, 2)
sequence_set.update(double_mismatch_sequences)
log.info('Double mismatch seqs: %d' % len(double_mismatch_sequences))
double_deletions = fbseqtools.get_deletion_seqs(target, 2)
log.info('Double deletion seqs: %d' % len(double_deletions)) #gives the text a red background
sequence_set.update(double_deletions)

#double_insertion_sequences = fbseqtools.get_insertion_seqs(target, 2)
#Change code to limit sequences. 
double_insertion_sequences = list(fbseqtools.get_insertion_seqs(target[3:], 2))
for i,sequence in enumerate(double_insertion_sequences): 
    double_insertion_sequences[i]=target_pam[:3]+sequence
double_insertion_sequences = set(double_insertion_sequences)
#End code change

log.info('Double insertion seqs: %d' % len(double_insertion_sequences))
sequence_set.update(double_insertion_sequences)

Interesting PAM seqs: 1
Single mismatch seqs: 81
Single insertion seqs: 82
Single deletion seqs: 20
Complement stretch seqs: 351
Double mismatch seqs: 3159
Double deletion seqs: 191
Double insertion seqs: 2622


In [13]:
#Generate complete sequences from collected sequences 
complete_sequences = set()
for sequence in sequence_set:
    left_barcode, right_barcode = barcode_pairs.pop()
    complete_sequences.add(NucleaSeqOligo(cr1, left_barcode, buffer1, sequence, buffer2, '', right_barcode, cr2))
    if len(complete_sequences) == total_desired_seqs - n_err_detect_seqs - min_perfect_target_copies: break 

In [14]:
# Perfect target with different buffer regions    
alt_buffers = set([(buffer, buffer[::-1]) for buffer in fbseqtools.get_randomized_stretch_seqs(buffer1, 2)
               if 'AAA' not in buffer and 'CCC' not in buffer and 'GGG' not in buffer and 'TTT' not in buffer
               and buffer.startswith(buffer1[:3])])


#What portion of the DNA is made up of G's and C's
good_gc_alt_buffers = set()
for a, b in alt_buffers:
    gc_content = float(a.count('C') + a.count('G')) / len(a)
    if gc_content <= 0.6:
        good_gc_alt_buffers.add((a, b))

for left_buffer, right_buffer in good_gc_alt_buffers:
    left_barcode, right_barcode = barcode_pairs.pop()
    complete_sequences.add(
        NucleaSeqOligo(cr1, 
                       left_barcode,
                       left_buffer,
                       target, 
                       right_buffer, 
                       '', 
                       right_barcode, 
                       cr2))
#    log.info('Alternative buffer with perfect target seqs: %d' % len(good_gc_alt_buffers))

# Perfect target with various barcodes
#    log.info('Perfect target with different barcode seqs: %d' % min_perfect_target_copies)

In [15]:
for _ in range(min_perfect_target_copies):
    left_barcode, right_barcode = barcode_pairs.pop()
    complete_sequences.add(NucleaSeqOligo(cr1, left_barcode, buffer1, target, buffer2, '', right_barcode, cr2))


# Error rate detection sequences    
len_err_detect_seq = (
    max(len(seq) for seq in complete_sequences) 
    - sum(len(s) for s in [cr1, left_barcode, '', right_barcode, cr2])
)
err_detect_seqs = generate_clean_random_eqmarg_seqs(nseq=n_err_detect_seqs,
                                                seqlen=len_err_detect_seq)
err_min_dists = [min_dist_to_target_seed(seq, target_seeds) for seq in err_detect_seqs]
#    log.info('Before search min error detection seq dist to target seed quartiles: {} / {} / {} / {} / {}'.format(
#        *map(int, np.percentile(err_min_dists, [0, 25, 50, 75, 100]))
#   ))


for _ in range(50):
    test_err_detect_seqs = generate_clean_random_eqmarg_seqs(nseq=n_err_detect_seqs,
                                                         seqlen=len_err_detect_seq)
    if (min(min_dist_to_target_seed(seq, target_seeds) for seq in test_err_detect_seqs)
        > min(min_dist_to_target_seed(seq, target_seeds) for seq in err_detect_seqs)):
        err_detect_seqs = test_err_detect_seqs
        
err_min_dists = [min_dist_to_target_seed(seq, target_seeds) for seq in err_detect_seqs]
#    log.info('After search min error detection seq dist to target seed quartiles: {} / {} / {} / {} / {}'.format(
#        *map(int, np.percentile(err_min_dists, [0, 25, 50, 75, 100]))
#    ))
#    log.info('Error detection seqs: %d' % len(err_detect_seqs))

KeyboardInterrupt: 

In [ ]:
for err_seq in err_detect_seqs:
    left_barcode, right_barcode = err_barcode_pairs.pop()
    complete_sequences.add(NucleaSeqOligo(cr1, left_barcode, '', err_seq, '', '', right_barcode, cr2))

In [ ]:
log.info('Alternative buffer with perfect target seqs: %d' % len(good_gc_alt_buffers))
log.info('Perfect target with different barcode seqs: %d' % min_perfect_target_copies)
log.info('Before search min error detection seq dist to target seed quartiles: {} / {} / {} / {} / {}'.format(
    *map(int, np.percentile(err_min_dists, [0, 25, 50, 75, 100]))))
log.info('After search min error detection seq dist to target seed quertiles: {} / {} / {} / {}/ {}'.format(
    *map(int, np.percentile(err_min_dists, [0, 25, 50, 75, 100]))))
log.info('Error detection seqs: %d' % len(err_detect_seqs))

In [16]:
#----------------------------------------------------------------------------------
# Fill unused seqs
#----------------------------------------------------------------------------------


#Will take time to complete

log.info('Remaining seqs to be filled: {}'.format(total_desired_seqs - len(complete_sequences)))
num_singles = len(single_deletions) + len(single_insertions) + len(single_mismatches)
singleton_copies = 1
singleton_copies += 1
for sequence in single_deletions | single_insertions | single_mismatches:
    left_barcode, right_barcode = barcode_pairs.pop()
    complete_sequences.add(NucleaSeqOligo(cr1, left_barcode, buffer1, sequence, buffer2, '', right_barcode, cr2))
    added_perfects = 0
    added_perfects += 1
    left_barcode, right_barcode = barcode_pairs.pop()
    complete_sequences.add(NucleaSeqOligo(cr1, left_barcode, buffer1, target, buffer2, '', right_barcode, cr2))

    
log.info('Added perfect seqs: {}'.format(added_perfects))
log.info('Total perfect seqs: {}'.format(added_perfects 
                                              + min_perfect_target_copies
                                              + len(good_gc_alt_buffers)))
log.info('Total copies/sequences of single error seqs: {}/{}'.format(
    singleton_copies,
    singleton_copies*num_singles
))    
log.info('Total sequences generated: %d' % len(complete_sequences))
if len(barcode_pairs) > 0:
    log.warning('\n(%d unused barcode pairs)\n' % len(barcode_pairs))

Remaining seqs to be filled: 5929
Added perfect seqs: 1
Total perfect seqs: 63
Total copies/sequences of single error seqs: 2/366
Total sequences generated: 6909

(5413 unused barcode pairs)



In [ ]:
oligo = list(complete_sequences)[0]
print 'Example oligo ({} bp):'.format(len(oligo))
oligo.pieces

In [ ]:
bad_gc_seqs = [seq for seq in complete_sequences if seq.gc_content >= 0.6 or seq.gc_content <= 0.4]
print '{:.1f}% of sequences have unusually high or low GC content'.format(
    100*float(len(bad_gc_seqs))/len(complete_sequences))

In [ ]:
left_bcs = set(oligo._barcode_left for oligo in complete_sequences)
right_bc_rcs = set(str(Seq(oligo._barcode_right).reverse_complement()) for oligo in complete_sequences)
assert len(left_bcs) == len(right_bc_rcs)
print 'Number of seqs, barcodes:', len(left_bcs)

In [17]:
raw_oligo_lens = set(map(len, complete_sequences))
print 'Raw oligos range in length from {} nt to {} nt'.format(min(raw_oligo_lens), max(raw_oligo_lens))

Raw oligos range in length from 109 nt to 113 nt


# Build Primers

In [ ]:
pamtarg_coord_one_pos = len(target_pam)
random.seed(rand_seeds[9])
complete_sequences = design.update_buffers(
    complete_sequences, 
    min_primer_len, 
    min_buffer_len,
    target,
    bad_substrs,
    fudge_factor,
    abs_cannonical_cut_sites,
    pamtarg_coord_one_pos
)

print 'Oligo lengths:', set(map(len, complete_sequences))

In [ ]:
oligo = list(complete_sequences)[0]
print 'Example oligo:'
oligo.pieces

## Find usable primer prefixes (This step is RAM heavy)

In [ ]:
for primer_len in range(min_primer_len, max_primer_len + 1):
    print 'Trying', primer_len
    random.seed(rand_seeds[3])
    complete_sequences = design.update_buffers(
        complete_sequences, 
        primer_len, 
        min_buffer_len,
        target,
        bad_substrs,
        fudge_factor,
        abs_cannonical_cut_sites,
        pamtarg_coord_one_pos
    )
    print 'Oligo lengths:', set(map(len, complete_sequences))
    oligo = list(complete_sequences)[0]
    print 'Left/right buffers: {} / {}'.format(oligo._buffer_left, oligo._buffer_right)
    
    good_prefixes = design.find_good_prefixes(
        complete_sequences, 
        primer_len, 
        bad_substrs,
        abs_cannonical_cut_sites,
        fudge_factor,
        nprocs
    )
    if len(good_prefixes) >= 2:
        break
    print 

assert len(good_prefixes) >= 2
print
print 'Success!'
print good_prefixes
print

if primer_len == min_primer_len:
    prev_good_prefixes = good_prefixes[:]
    for fake_primer_len in range(primer_len-1, 0, -1):
        print 'Trying buffers based on', fake_primer_len
        random.seed(rand_seeds[3])
        complete_sequences = design.update_buffers(
            complete_sequences, 
            fake_primer_len, 
            min_buffer_len,
            target,
            bad_substrs,
            fudge_factor,
            abs_cannonical_cut_sites,
            pamtarg_coord_one_pos
        )
        print 'Oligo lengths:', set(map(len, complete_sequences))
        oligo = list(complete_sequences)[0]
        print 'Left/right buffers: {} / {}'.format(oligo._buffer_left, oligo._buffer_right)

        good_prefixes = design.find_good_prefixes(
            complete_sequences, 
            primer_len, 
            bad_substrs,
            abs_cannonical_cut_sites,
            fudge_factor,
            nprocs
        )
        if len(good_prefixes) >= 2:
            print
            print good_prefixes
            prev_good_prefixes = good_prefixes[:]
        else:
            break
        print 
    print
    print 'Failed, reverting to best previous:'
    good_prefixes = prev_good_prefixes
    fake_primer_len += 1
print good_prefixes

# Replace primers

Test the above primers in your oligo analyzer of choice. If your experimental conditions require a longer primer, add bases as desired below. If you need shorter primers, do not change anything here. The full length of these primers is needed for the post-experiment analysis. Simply order primers with the inner bases removed as needed for your experiment.

In [ ]:
left_primer = good_prefixes[0] + ''
right_primer = good_prefixes[1] + 'G'
cr_left = left_primer
cr_right = fbseqtools.dna_rev_comp(right_primer)

print 'Left / Right primers:'
print left_primer
print right_primer
print
print 'Hence, the oligos will have the form:'
print
print '{}    BC_left/Buf_left/Target/Buf_right/BC_right*    {}'.format(cr_left, cr_right)

In [ ]:
for oligo in complete_sequences:
    oligo._cr_left = cr_left
    oligo._cr_right = cr_right

In [ ]:
print 'Oligo lengths:', set(map(len, complete_sequences))

# Output

In [ ]:
seq_ffolder = '/home/joneslab/Lab_Data/'
seq_ffile ='target_{}_TnpB_Buffered_seqs.txt'.format(target_name)
seq_fpath = seq_ffolder+seq_ffile
exploded_seq_fpath = seq_ffolder+'exploded_' + seq_ffile

with open(seq_fpath, 'w') as out:
    out.write('\n'.join([oligo.sequence for oligo in complete_sequences]))
with open(exploded_seq_fpath, 'w') as out:
    out.write('\n'.join([oligo.tab_delimited_str() for oligo in complete_sequences]))

In [ ]:
total_time = time.time() - notebook_start_time

print 'Run time: {} seconds'.format(total_time)